In [45]:
%pip install arxiv


Note: you may need to restart the kernel to use updated packages.


In [29]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [30]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)


In [31]:
wiki.name

'wikipedia'

In [32]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

ollama_embeddings = OllamaEmbeddings(base_url='http://localhost:11434/', model='llama3.2')

vectordb=FAISS.from_documents(documents[:30],ollama_embeddings)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000015AE06EE990>, search_kwargs={})

In [33]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith_search","Search for information about LangSmith. For any information about LangSmith, please use this tool.")

In [34]:
retriever_tool.name

'langsmith_search'

In [35]:
### Arxiv Tool
from langchain_community.utilities.arxiv import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [36]:
tools = [wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\dev\\Learning\\Langchain\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any information about LangSmith, please use this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000015AECE2C9A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_communit

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

from langchain_openai import ChatOpenAI
llm_openai = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)

# THE FOLLOWING IS NOT WORKING AS I'M USING "hwchase17/openai-functions-agent" prompts which only works with the "openai" package
# from langchain_community.llms import Ollama
# llm_ollama = Ollama(model='llama3.2', base_url='http://localhost:11434/', temperature=0)


In [38]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [51]:
## Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm_openai,tools,prompt)

In [52]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [53]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Get started by adding tracing to your application.
Create dashboards to view key metrics like RPS, error rates and costs.

Evals​
The quality and development speed of AI applications depends on high-quality evaluation datasets and metrics to test and optimize your applications on. The LangSmith SDK and UI make building and running high-quality evaluations easy.

Get started by creating your first evaluation.
Quickly assess the performance of your application using our off-the-shelf evaluators as a starting point.
Analyze results of evaluations in the LangSmith UI and compare results over time.
Easily collect human feedback on your data to improve your application.

Prompt Engineering​
While traditional software applications are built by writing code, AI applications involve writing prompts to instruct the LLM on what to do. LangSmith provide

{'input': 'Tell me about Langsmith',
 'output': "LangSmith is a platform that offers tools and features for building and optimizing AI applications. Here are some key aspects of LangSmith:\n\n1. **Evals**: LangSmith provides tools for creating and running high-quality evaluations to test and optimize AI applications. Users can analyze evaluation results in the LangSmith UI and collect human feedback to improve their applications.\n\n2. **Prompt Engineering**: In AI applications, writing prompts is essential for instructing the Language Model (LLM) on what to do. LangSmith offers tools for prompt engineering to help users find the perfect prompt for their applications.\n\n3. **Observability**: Observability is crucial for LLM applications as they are non-deterministic and can produce unexpected results. LangSmith provides LLM-native observability features to help users gain meaningful insights throughout all stages of application development.\n\nLangSmith is framework-agnostic and can b

In [54]:
agent_executor.invoke({"input":"What's the paper 2404.11170 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '2404.11170'}`


Published: 2024-09-22
Title: Relating bubble sort to birthday problem
Authors: Jichu Jiang
Summary: Birthday problem is a well-known classic problem in probability theory widely
applied in cryptographThe paper with identifier 2404.11170 is titled "Relating bubble sort to birthday problem" by Jichu Jiang. It discusses the relationship between bubble sort and the birthday problem, a classic problem in probability theory widely applied in cryptography.

> Finished chain.


{'input': "What's the paper 2404.11170 about?",
 'output': 'The paper with identifier 2404.11170 is titled "Relating bubble sort to birthday problem" by Jichu Jiang. It discusses the relationship between bubble sort and the birthday problem, a classic problem in probability theory widely applied in cryptography.'}

In [55]:
agent_executor.invoke({"input":"Tell me about Machine Learning."})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. It involves creating models that can analyze and make decisions or predictions based on data without being explicitly programmed to do so. Machine learning is used in various applications such as image recognition, natural language processing, and recommendation systems.

> Finished chain.


{'input': 'Tell me about Machine Learning.',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. It involves creating models that can analyze and make decisions or predictions based on data without being explicitly programmed to do so. Machine learning is used in various applications such as image recognition, natural language processing, and recommendation systems.'}